## Creation of database with parent-comment pairs from Reddit JSON

In [ ]:
#loading all the required packages
import sqlite3
import json
import pandas as pd

In [5]:
#creating the db file and reddit comment(rc) table
db_file=r'reddit_comments.db'
conn = sqlite3.connect(db_file)
cur=conn.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS rc (
parent_id TEXT PRIMARY KEY,
comment_id TEXT UNIQUE,
pcomment TEXT,
comment TEXT,
score INT,
subreddit TEXT)""")

## All the required helper functions

In [118]:
transact=[]
def time_saver(sql):
    global transact
    transact.append(sql)
    if len(transact)>1000:
        cur.execute("""BEGIN TRANSACTION""")
        for t in transact:
            try:
                cur.execute(t)
            except Exception as e:
                #print(e)
                #print(sql)
                pass
        conn.commit()
        transact=[]

def parent_exists(pid):
    try:
        cur.execute("SELECT comment from rc WHERE comment_id='{}'".format(pid))
        rows=cur.fetchone()
        if rows==None:
            return False
        return rows[0]
    except:
        return False

def accept(comment):
    if len(comment)>1000:
        return False
    elif len(comment.split(' '))>50 or len(comment.split(' '))<1:
        return False
    elif comment=='[deleted]' or comment=='[removed]':
        return False
    else:
        return True

def score_exists(pid):
    cur.execute("SELECT score from rc WHERE parent_id = '{}'".format(pid))
    rows=cur.fetchone()
    if rows == None:
        return False
    return rows[0]
    
def update_score_exists(cid,body,rank,pid):
    sql="""UPDATE rc SET comment_id="{}",comment="{}",score={} WHERE 
    parent_id='{}'""".format(cid,body,rank,pid)
    time_saver(sql)

def insert_parent_exists(cid,body,sred,pid,pbody,rank):
    sql="""INSERT INTO rc (comment_id,comment,subreddit,parent_id,pcomment,score) VALUES
    ("{}","{}","{}","{}","{}",{})""".format(cid,body,sred,pid,pbody,rank)
    time_saver(sql)

def insert_no_parent(cid,body,sred,pid,rank):
    sql="""INSERT INTO rc (comment_id,comment,subreddit,parent_id,score) VALUES 
    ("{}","{}","{}","{}",{})""".format(cid,body,sred,pid,rank)
    time_saver(sql)

In [131]:
cur.execute("""DELETE FROM rc""")
cur.execute("""SELECT * FROM rc""")
print(cur.rowcount)

-1


In [132]:
row_count=0
pairs=0
check1=False
check2=False
chaeck3=False
with open("/home/harish_gunda16/RC_2015-01") as file:
    for line in file:
        row_count+=1
        line=json.loads(line)
        cid=line['name']
        body=line['body'].replace('\n',' ').replace('"',"'")
        rank=line['score']
        sred=line['subreddit']
        pid=line['parent_id']
        if rank>2:
            if accept(body):
                curr_rank=score_exists(pid)
                if curr_rank!=False:
                    if rank>curr_rank:
                        update_score_exists(cid,body,rank,pid)
                        check1=True
                pbody=parent_exists(pid)
                if pbody!=False:
                    insert_parent_exists(cid,body,sred,pid,pbody,rank)
                    check2=True
                    pairs+=1
                else:
                    insert_no_parent(cid,body,sred,pid,rank)
                    check3=True
        if (row_count%5000000)==0:
            print('rows processed=',row_count)
            print('pairs generated=',pairs)

rows processed= 5000000
pairs generated= 228443
rows processed= 10000000
pairs generated= 468548
rows processed= 15000000
pairs generated= 708503
rows processed= 25000000
pairs generated= 1184987
rows processed= 30000000
pairs generated= 1424514
rows processed= 35000000
pairs generated= 1660413
rows processed= 40000000
pairs generated= 1898777
rows processed= 50000000
pairs generated= 2373341


In [151]:
pairs

2557199

In [154]:
input=pd.read_sql("SELECT * FROM rc WHERE parent_id IS NOT NULL LIMIT 100",conn)

In [153]:
input.head()

,parent_id,comment_id,pcomment,comment,score,subreddit
0,t3_2r0rom,t1_cnbbfee,None,"Tumblr, not even once. It's worse than meth wh...",95,TumblrInAction
1,t3_2r1ruk,t1_cnbnb5u,None,When I run the same play twice in a row in NCA...,1024,CFB
2,t1_cnbkf1f,t1_cnbn4j0,"Yeah, probably.","If she can't say yes on the spot, I'd legit re...",11,AskMen
3,t1_cnbn0v1,t1_cnbn3nz,None,Oh. Now I understand what he means.,7,todayilearned
4,t1_cnbmr7j,t1_cnbn4j5,None,Kim Jong Un is in Australia?,13,gifs
